In [1]:
from keras.applications import ResNet50
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

def Resnet50_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # validation split

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10,
                                                                    stratify=y_train)
    print(len(X_train))


    model = ResNet50(include_top=True, weights=None, input_shape=(227,227,3), pooling=max, classes=4)
    model.compile(loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'],
                  optimizer=keras.optimizers.SGD(learning_rate=0.01))
    modelhistory = model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_validation, y_validation))
    evaluation = []
    evaluation.append(model.evaluate(X_train, y_train, batch_size=16))
    evaluation.append(model.evaluate(X_test, y_test, batch_size=16))
    
    return evaluation

In [ ]:
image_size = 227

groups_folder_path = '../Data/AAFT_5/'

X_s = []

y = []
# auto_corr       hadamard        midAng     striping_color      striping_midAng_color
for label in ['Bi', 'Tri', 'Hammer', 'Rvcurl']:

    for feature in ['striping_color']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            #print(f)
            for filename in f:
                #print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])
                
                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                elif label == 'Tri':
                    y.append(1)
                elif label == 'Hammer':
                    y.append(2)
                else:
                    y.append(3)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

res = []
for i in range(2):
    res.append(Resnet50_model(X, y))

(227, 227, 3)
1200
864
Epoch 1/10
54/54 [==============================] - 20s 144ms/step - loss: 5.3163 - accuracy: 0.2745 - val_loss: 1.6254 - val_accuracy: 0.2500
Epoch 2/10
54/54 [==============================] - 7s 129ms/step - loss: 1.5429 - accuracy: 0.5828 - val_loss: 2.9531 - val_accuracy: 0.2500
Epoch 3/10
54/54 [==============================] - 7s 129ms/step - loss: 0.9056 - accuracy: 0.7418 - val_loss: 4.3125 - val_accuracy: 0.2500
Epoch 4/10
54/54 [==============================] - 7s 129ms/step - loss: 0.8150 - accuracy: 0.7585 - val_loss: 6.1805 - val_accuracy: 0.2500
Epoch 5/10
54/54 [==============================] - ETA: 0s - loss: 0.4211 - accuracy: 0.8463

In [ ]:
for data in res:
    data[0][0]

In [ ]:
for data in res:
    data[1][1]